## HyperDT vs sklearn with MNIST data

In [2]:
%load_ext autoreload
%autoreload 2

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from hyperdt.tree import HyperbolicDecisionTreeClassifier

### Load MNIST dataset

In [4]:
mnist = fetch_openml('mnist_784')

/opt/miniconda3/envs/hyperDT/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [23]:
# Convert pandas dataframe to numpy arrays
X = mnist.data.to_numpy()
y = mnist.target.to_numpy().astype(int)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=42)
print(X_train.shape, X_test.shape)

(60000, 784) (10000, 784)


In [30]:
# Add time dimension as Euclidean distance of all other dimensions
K = 1
train_x_0 = np.sqrt(np.sum(X_train**2, axis=1) + 1/K).reshape(-1, 1)
test_x_0 = np.sqrt(np.sum(X_test**2, axis=1) + 1/K).reshape(-1, 1)
X_train_time = np.concatenate([train_x_0, X_train], axis=1)
X_test_time = np.concatenate([test_x_0, X_test], axis=1)
print(X_train_time.shape,X_test_time.shape)

(60000, 785) (10000, 785)


### sklearn decision tree model in Euclidean space

In [31]:
# Train decision tree classifier
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [32]:
# Predict test set labels
y_pred = dtc.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.87


### HyperDT on augmented MNIST data

In [33]:
def Minkowski(X):
    return np.sum(X[:, 1:]**2, axis=1) - X[:, 0]**2

In [36]:
# Fit hyperbolic decision tree classifier
hdt = HyperbolicDecisionTreeClassifier(timelike_dim=0, max_depth=2)
hdt.fit(X_train_time, y_train)

# Compute accuracy
hdt_score = hdt.score(X_test_time, y_test)
print(hdt_score)

dim: 13, theta: 1.5405136975122538, count: 0
dim: 13, theta: 1.5685317496548659, count: 1
dim: 14, theta: 1.4609818288556855, count: 0
dim: 14, theta: 1.5218032660830223, count: 1
dim: 15, theta: 1.5218032660830223, count: 0
dim: 16, theta: 1.5687582086928242, count: 0
dim: 33, theta: 1.5675345082246002, count: 0
dim: 34, theta: 1.5514473965984195, count: 0
dim: 34, theta: 1.5596684215055525, count: 1
dim: 34, theta: 1.5692502024051438, count: 2
dim: 35, theta: 1.4799442285909108, count: 0
dim: 35, theta: 1.5152346310896552, count: 1
dim: 35, theta: 1.5286949937547991, count: 2
dim: 35, theta: 1.5411410689187859, count: 3
dim: 35, theta: 1.5576363606147217, count: 4
dim: 35, theta: 1.5704527557183448, count: 5
dim: 36, theta: 1.4723837224558052, count: 0
dim: 36, theta: 1.4818216317367712, count: 1
dim: 36, theta: 1.4922625283770627, count: 2
dim: 36, theta: 1.4967199627539662, count: 3
dim: 36, theta: 1.5173814592415122, count: 4
dim: 36, theta: 1.5375584124033217, count: 5
dim: 36, t

KeyboardInterrupt: 